In [ ]:
!pip install transformers

In [1]:
import argparse
import json
import numpy as np
import os
import pickle
import random
import time
import tqdm
import subprocess
import sys
from typing import List

In [2]:
from transformers import AdamW, get_linear_schedule_with_warmup, AutoModelForSequenceClassification, AutoTokenizer
from transformers.trainer import set_seed

In [3]:
import torch
from torch.nn import CrossEntropyLoss

In [4]:
sys.path.append('..')

In [5]:
from ctp.metrics import compute_metrics, compute_scores, flat_accuracy
from ctp.utils import (
    check_backwards_compatability,
    format_time,
    get_device,
    get_sentences_results_wordnet_df,
    print_average_metrics,
    str2bool,
)
from ctp.inference.examine_subtrees import run_inference_subtree
from datasets.model_inputs.dataset import get_dataset

device = get_device()

Using GPU: Tesla T4


In [6]:
print(1)

1


In [7]:
def set_random_seed(seed_val):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)
    set_seed(seed_val)

In [8]:
def run_inference(test_filename: str,
        epoch_num: int,
        results_filename: str,
        subtrees_filename: str,
        results_dir: str,
        data_dir: str,
        wordnet_dir: str = './datasets/data_creators/df_csvs/',
        prediction_metric_type: str = 'ancestor',
        wordnet_filename: str = "bansal14_trees.csv",
        wordnet_df_names: List[str] = ['hypernym', 'term', 'tree_id', 'train_test_split'],
        keys_to_print: List[str] = ['pruned_precision', 'pruned_recall', 'pruned_f1']):

    sentences, results, wordnet_df = get_sentences_results_wordnet_df(
            wordnet_filepath=os.path.join(wordnet_dir, wordnet_filename),
            results_filepath=os.path.join(results_dir, results_filename.format(epoch_num=epoch_num)),
            sentences_filepath=os.path.join(data_dir, test_filename))
    tree_ids = np.unique([val["tree_id"] for val in sentences.values()])

    subtrees_dict = {}
    subtrees_info_dict = {}

    for tree_id in tree_ids:
        run_inference_subtree(
            tree_id=tree_id,
            prediction_metric_type=prediction_metric_type,
            sentences=sentences,
            results=results,
            subtrees_dict=subtrees_dict,
            subtrees_info_dict=subtrees_info_dict,
            wordnet_df=wordnet_df,
        )

    print_average_metrics(subtrees_info_dict, epoch_num)

    with open(os.path.join(results_dir, subtrees_filename.format(epoch_num=epoch_num)), 'wb') as f:
        pickle.dump(subtrees_dict, f)

In [9]:
def run_validation(test_data,
        test_filename,
        train_filename,
        model,
        results_dir,
        epoch_num,
        experiment_name,
        results_filename,
        metrics_filename):
    """
    test_data: (test_data, test_hashes).
    """
    test_dataloader, test_hashes = test_data
    eval_accuracy = 0
    nb_eval_steps = 0
    results_dict = {}
    hash_index = 0  # TODO: Better way to associate hash with example?
    t0 = time.time()
    for batch_num, batch in enumerate(tqdm.tqdm(test_dataloader)):
        batch = tuple(t.to(device) for t in batch)

        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            outputs = model(input_ids=b_input_ids,
                    token_type_ids=None,
                    attention_mask=b_input_mask)[0]

        logits = outputs

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to("cpu").numpy()
        for input_id, logit, label_id in zip(b_input_ids, logits, label_ids):
            example_id = test_hashes[hash_index]
            hash_index += 1
            results_dict[example_id] = {
                "logits": logit.tolist(),
                "label": [int(label_id)],
            }

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)

        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print(f"Validation Accuracy: {round(eval_accuracy / nb_eval_steps, 2)}")
    print(f"Validation took: {format_time(time.time() - t0)}")

    with open(os.path.join(results_dir, results_filename.format(epoch_num=epoch_num)), "w") as f:
        json.dump(results_dict, f)

    metrics_dict = compute_scores(results_dict=results_dict)
    print("  metrics", metrics_dict)
    with open(os.path.join(results_dir, metrics_filename.format(epoch_num=epoch_num)), "w") as f:
        json.dump(metrics_dict, f)

    return eval_accuracy / nb_eval_steps

In [10]:
def train(
    train_filename,
    test_filenames,
    subtrees_filename,
    data_dir,
    results_dir,
    ckpt_dir,
    results_filename,
    ckpt_filename,
    metrics_filename,
    test_wordnet_filenames,
    batch_size,
    num_epochs,
    learning_rate,
    num_warmup_steps,
    num_labels,
    experiment_name,
    cased,
    reload_from_epoch_num,
    validate_only,
    model,
    reload_from_checkpoint,
    max_len,
    dataset_type='sequence_classification',
    adam_eps=1e-8,
    seed_val=2020,
):
    tokenizer = AutoTokenizer.from_pretrained(model)
    model = AutoModelForSequenceClassification.from_pretrained(model)

    if reload_from_checkpoint:
        ckpt_filename = os.path.join(ckpt_dir, ckpt_filename.format(epoch_num=reload_from_epoch_num))
        print(f"reloading from {ckpt_filename}")
        model.load_state_dict(torch.load(ckpt_filename))
        starting_epoch_num = int(reload_from_epoch_num.split("_")[0])
    else:
        starting_epoch_num = -1

    t0 = time.time()

    if not args.validate_only:
        print(f"{format_time(time.time()-t0)} Getting train dataset")
        train_dataloader, _ = get_dataset(train_filename,
                data_dir,
                tokenizer=tokenizer,
                batch_size=batch_size,
                max_len=max_len,
                dataset_type=dataset_type,
                is_train_data=True)
        num_training_steps = num_epochs * len(train_dataloader)
        first_epoch_validation_increment = len(train_dataloader) // 10
        optimizer = AdamW(model.parameters(), lr=learning_rate, eps=adam_eps)
        scheduler = get_linear_schedule_with_warmup(optimizer,
                num_warmup_steps=num_warmup_steps,
                num_training_steps=num_training_steps)

    print(f"{format_time(time.time()-t0)} Getting validation dataset")
    test_datas_list = [get_dataset(test_filename,
        data_dir,
        tokenizer=tokenizer,
        batch_size=batch_size,
        max_len=max_len,
        dataset_type=dataset_type,
        is_train_data=False)
        for test_filename in test_filenames]

    model.to(device)

    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    loss_values = []

    print(f"{format_time(time.time()-t0)} Begin training")

    def do_validation():
        print("")
        print("Running Validation...")
        for test_data, test_filename, test_wordnet_filename in zip(test_datas_list, test_filenames, test_wordnet_filenames):
            run_validation(test_data=test_data,
                    test_filename=test_filename,
                    train_filename=train_filename,
                    model=model,
                    results_dir=results_dir,
                    epoch_num=epoch_num,
                    experiment_name=experiment_name,
                    results_filename=results_filename,
                    metrics_filename=metrics_filename)

            run_inference(test_filename=test_filename,
                    epoch_num=epoch_num,
                    results_filename=results_filename,
                    subtrees_filename=subtrees_filename,
                    results_dir=results_dir,
                    wordnet_filename=test_wordnet_filename,
                    data_dir=data_dir)

    if validate_only:
        epoch_num = reload_from_epoch_num
        do_validation()
        return

    for epoch_num in range(starting_epoch_num + 1, starting_epoch_num + 1 + num_epochs):
        print(f"Epoch num: {epoch_num}")
        do_validation()
        t0 = time.time()
        total_loss = 0
        train_accuracy = 0
        nb_train_steps = 0
        model.train()
        for step, batch in enumerate(tqdm.tqdm(train_dataloader)):
            if step % 40 == 0 and not step == 0:
                elapsed = format_time(time.time() - t0)
                print(f"  Batch {step}  of  {len(train_dataloader)}.    Elapsed: {elapsed}.")

            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            model.zero_grad()
            outputs = model(b_input_ids,
                    token_type_ids=None,
                    attention_mask=b_input_mask)[0]
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(outputs.view(-1, num_labels), b_labels.view(-1))
            total_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

            logits = outputs
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to("cpu").numpy()
            tmp_train_accuracy = flat_accuracy(logits, label_ids)
            train_accuracy += tmp_train_accuracy
            nb_train_steps += 1

            if (epoch_num == 0
                    and step % first_epoch_validation_increment == 0
                    and not step == 0):
                print(f"    Step {step}, epoch {epoch_num}")
                do_validation()
                torch.save(model.state_dict(),
                        os.path.join(ckpt_dir, ckpt_filename.format(epoch_num=epoch_num)).replace('.ckpt', f'_step_{step}.ckpt'))
                compute_metrics(labels=label_ids, label_predictions=np.argmax(logits, axis=-1))

        avg_train_loss = total_loss / len(train_dataloader)
        loss_values.append(avg_train_loss)

        print(f"  Training Accuracy: {round(train_accuracy / nb_train_steps, 2)}")
        print(f"  Average training loss: {round(avg_train_loss, 2)}")
        print(f"  Training epoch took: {format_time(time.time() - t0)}")
        torch.save(model.state_dict(),
            os.path.join(ckpt_dir, ckpt_filename.format(epoch_num=epoch_num)))
        compute_metrics(labels=label_ids, label_predictions=np.argmax(logits, axis=1))

        t0 = time.time()

        model.eval()
    print("")
    print("Training complete!")

In [16]:
from dataclasses import dataclass
@dataclass
class Arguments():
    train_filename: str="wordnet_bansal_anc_0_sib_0_desc_0_rand_0_parent_1_train_subsample_0_pos_subset_None.json"
    test_filenames: list=("wordnet_bansal_anc_0_sib_0_desc_0_rand_0_parent_1_test_subsample_0_pos_subset_None.json",)
    subtrees_filename: str="subtrees_par_bert_1e6_seed2.json"
    data_dir: str = "./datasets/generated_training_pairs"
    results_dir: str="./outputs/results/"
    ckpt_dir: str="./outputs/ckpts/"
    results_filename: str="logits_par_bert_1e6_seed2.json"
    metrics_filename: str="metrics_par_bert_1e6_seed2.json"
    test_wordnet_filenames: list=("bansal14_trees.csv",)
    ckpt_filename: str="ckpt_par_bert_1e6_seed2.json"
    batch_size: int=32
    num_epochs: int=10
    learning_rate: float=1e-5
    num_warmup_steps: int=320
    num_labels: int=2
    experiment_name: str="WN_par_bert_1e6_seed2"
    cased: bool= 0
    reload_from_epoch_num="10_0"
    validate_only: bool=1
    reload_from_checkpoint: bool=1
    model: str="bert-base-uncased"
    max_len: int=64
    dataset_type: str="sequence_classification"
    random_seed: int=0

args = Arguments()
args

Arguments(train_filename='wordnet_bansal_anc_0_sib_0_desc_0_rand_0_parent_1_train_subsample_0_pos_subset_None.json', test_filenames=('wordnet_bansal_anc_0_sib_0_desc_0_rand_0_parent_1_test_subsample_0_pos_subset_None.json',), subtrees_filename='subtrees_par_bert_1e6_seed2.json', data_dir='./datasets/generated_training_pairs', results_dir='./outputs/results/', ckpt_dir='./outputs/ckpts/', results_filename='logits_par_bert_1e6_seed2.json', metrics_filename='metrics_par_bert_1e6_seed2.json', test_wordnet_filenames=('bansal14_trees.csv',), ckpt_filename='ckpt_par_bert_1e6_seed2.json', batch_size=32, num_epochs=10, learning_rate=1e-05, num_warmup_steps=320, num_labels=2, experiment_name='WN_par_bert_1e6_seed2', cased=0, validate_only=1, reload_from_checkpoint=1, model='bert-base-uncased', max_len=64, dataset_type='sequence_classification', random_seed=0)

In [19]:
# %cd ..

In [25]:
check_backwards_compatability(args)
print(f"args {args}")

if not os.path.exists("./outputs/logs"):
    os.makedirs("./outputs/logs")

if not os.path.exists(args.ckpt_dir):
    os.makedirs(args.ckpt_dir)

if not os.path.exists(args.results_dir):
    os.makedirs(args.results_dir)

print('Experiment running on commit ', subprocess.check_output(['git', 'rev-parse', 'HEAD']))

set_random_seed(args.random_seed)
train(train_filename=args.train_filename,
      test_filenames=args.test_filenames,
      subtrees_filename=args.subtrees_filename,
      data_dir=args.data_dir,
      results_dir=args.results_dir,
      ckpt_dir=args.ckpt_dir,
      results_filename=args.results_filename,
      ckpt_filename=args.ckpt_filename,
      metrics_filename=args.metrics_filename,
      batch_size=args.batch_size,
      num_epochs=args.num_epochs,
      learning_rate=args.learning_rate,
      num_warmup_steps=args.num_warmup_steps,
      num_labels=args.num_labels,
      cased=args.cased,
      experiment_name=args.experiment_name,
      reload_from_checkpoint=args.reload_from_checkpoint,
      reload_from_epoch_num=args.reload_from_epoch_num,
      validate_only=args.validate_only,
      model=args.model,
      max_len=args.max_len,
      dataset_type=args.dataset_type,
      test_wordnet_filenames=args.test_wordnet_filenames)

args Arguments(train_filename='wordnet_bansal_anc_0_sib_0_desc_0_rand_0_parent_1_train_subsample_0_pos_subset_None.json', test_filenames=('wordnet_bansal_anc_0_sib_0_desc_0_rand_0_parent_1_test_subsample_0_pos_subset_None.json',), subtrees_filename='subtrees_par_bert_1e6_seed2.json', data_dir='./datasets/generated_training_pairs', results_dir='./outputs/results/', ckpt_dir='./outputs/ckpts/', results_filename='logits_par_bert_1e6_seed2.json', metrics_filename='metrics_par_bert_1e6_seed2.json', test_wordnet_filenames=('bansal14_trees.csv',), ckpt_filename='ckpt_par_bert_1e6_seed2.json', batch_size=32, num_epochs=10, learning_rate=1e-05, num_warmup_steps=320, num_labels=2, experiment_name='WN_par_bert_1e6_seed2', cased=0, validate_only=1, reload_from_checkpoint=1, model='bert-base-uncased', max_len=64, dataset_type='sequence_classification', random_seed=0)
Experiment running on commit  b'21a39b978b2c3a484627e608a17e02db1d7419a6\n'


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

reloading from ./outputs/ckpts/ckpt_par_bert_1e6_seed2.json
0:00:00 Getting validation dataset


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2301: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Using 54202 sentences from wordnet_bansal_anc_0_sib_0_desc_0_rand_0_parent_1_test_subsample_0_pos_subset_None.json
1694 examples loaded from wordnet_bansal_anc_0_sib_0_desc_0_rand_0_parent_1_test_subsample_0_pos_subset_None.json
0:00:10 Begin training

Running Validation...


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

100%|█████████▉| 1693/1694 [03:29<00:00,  8.86it/s]

	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 1694/1694 [03:29<00:00,  8.08it/s]


Validation Accuracy: 0.95
Validation took: 0:03:30
{'precision': 0.352, 'recall': 0.389, 'F1': 0.37, 'true_positives': 865, 'false_positives': 1590, 'false_negatives': 1360, 'true_negatives': 50387, 'accuracy': 0.946}
  metrics {'precision': 0.352, 'recall': 0.389, 'F1': 0.37, 'true_positives': 865, 'false_positives': 1590, 'false_negatives': 1360, 'true_negatives': 50387, 'accuracy': 0.946}
Epoch num 10_0: pruned_precision 0.54 // pruned_recall 0.53 // pruned_f1 0.53


In [40]:
with open("datasets/generated_training_pairs/wordnet_bansal_anc_0_sib_0_desc_0_rand_0_parent_1_train_subsample_0_pos_subset_None.json", "r") as f:
     data_dict = json.load(f)